In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance
 

In [3]:
def save_to_csv_yahoo(ticker, syear, smonth, sday, eyear, emonth, eday):
    start = dt.datetime(syear, smonth, sday)
    end = dt.datetime(eyear, emonth, eday)
    
    df = web.DataReader(ticker, 'yahoo', start, end)
    df.to_csv("./" + ticker + ".csv")
    return df

In [4]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv('./' + ticker + '.csv')
    except FileNotFoundError:
        print('File does not exist')
    else:
        return df
    

In [6]:
def add_daily_return_to_df(df, ticker):
    df['daily_return'] = (df['Adj Close'] / df['Adj Close'].shift(1))
    return df

In [7]:
def get_return_defined_time(df, syear, smonth, sday, eyear, emonth, eday):
    start = f"{syear}-{smonth}-{sday}"
    end = f"{eyear}-{emonth}-{eday}"
    mask = (df['Date'] >= start) & (df['Date'] <= end)
    daily_return = df.loc[mask]['daily_return'].mean()
    df2 = df.loc[mask]
    days = df2.shape[0]
    return (days * daily_return)

In [5]:
def download_multiple_stocks(syear, smonth, sday, eyear, emonth, eday, *args):
    for x in args:
        save_to_csv_yahoo(x, syear, smonth, sday, eyear, emonth, eday)

In [15]:
def define_risk(df):
    ma50 = 0
    for x in range(50):
        ma50 = df['Adj Close'].shift(x) + ma50
    df['ma50'] = ma50/50
    ma350 = 0
    for x in range(350):
        ma350 = df['Adj Close'].shift(x) + ma350
    df['ma350'] = ma350/350
    df['Risk'] = df['ma50']/df['ma350']
    
    
    max_value = df['Risk'].max()
    min_value = df['Risk'].min()
    df['Risk'] = (df['Risk'] - min_value) / (max_value - min_value)
    
    return df

In [10]:
def risk_below_level(risk,close):
#     import numpy as np
    signal   = []
    previous = -1.0
    for date,value in risk.iteritems():
        if value < 0.3 and previous >= 0.3:
            signal.append(close[date]*0.99)
        else:
            signal.append(np.nan)
        previous = value
    return signal

# Matplotlib Finance

In [16]:
def mplfinance_plot(df, title, chart_type, syear, smonth, sday, eyear, emonth, eday):
    start = f"{syear}-{smonth}-{sday}"
    end = f"{eyear}-{emonth}-{eday}"
#     df = get_df_from_csv(ticker)
    df.index = pd.DatetimeIndex(df['Date'])
    df_sub = df.loc[start:end]
    
#     mpf.plot(df_sub, type='candle', title='Candle')
#     mpf.plot(df_sub, type='line', title='Line')
#     mpf.plot(df_sub, type='ohlc', mav=20, title='OHLC')
    
    s = mpf.make_mpf_style(base_mpf_style='charles', rc={'font.size': 8})
    fig = mpf.figure(figsize=(12, 8), style=s) 
#     ax = fig.add_subplot(2,1,1)
#     av = fig.add_subplot(2,1,2, sharex=ax)
#     ax.set_title(title)
#     ax.addplot(extra_plot)
#     signal = risk_below_level(df_sub['Risk'], df_sub['Adj Close'])
    apds = [ mpf.make_addplot(df_sub['Risk'], secondary_y=True),
           mpf.make_addplot((df_sub['Risk'] * 0) + 0.2, secondary_y=True),
           mpf.make_addplot((df_sub['Risk'] * 0) + 0.4, secondary_y=True),
           mpf.make_addplot((df_sub['Risk'] * 0) + 0.6, secondary_y=True),
           mpf.make_addplot((df_sub['Risk'] * 0) + 0.8, secondary_y=True)]
#     apd = mpf.make_addplot(signal,type="scatter")
    
    hlines = dict(hlines=[0.2,0.8], colors=['g','r'], linestyle='-.')
    mpf.plot(df_sub, type=chart_type, mav=20, title=title, tight_layout=True, addplot=apds,
              volume=False, figscale=2, show_nontrading=True, style='charles', 
             hlines=hlines, )#panel_ratios=(3,1))

In [12]:
def plot_indicator(df,index):
    df.index = pd.DatetimeIndex(df['Date'])
    s = mpf.make_mpf_style(base_mpf_style='charles', rc={'font.size': 8})
    fig = mpf.figure(figsize=(12, 8),style=s)
    ax = fig.add_subplot(2,1,1)
    ax.set_title(index)
    mpf.plot(df, type='line', ax=ax, show_nontrading=True)

# Test Functions

In [4]:
symbol = 'MSFT'
# tickers = ['FB','AMZN','AAPL','NFLX','GOOGL', 
#            'SHLX','BABA','SPCE','AMD','BSX','GLP','DIS',
#            'MSFT','SPY','BTC-USD','ARKF']
# download_multiple_stocks(1999, 1, 1, 2021, 10, 20, *tickers)
# save_to_csv_yahoo(ticker, 1999, 1, 1, 2021, 10, 22 )
import yfinance
# df = yfinance.download(tickers=tickers, period="MAX")
# df.to_csv(ticker + ".csv")
ticker = yfinance.Ticker(symbol)
df = ticker.option_chain(ticker.options[0])
df[0]
# get_df_from_csv('NS')

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,MSFT211029C00145000,2021-10-18 13:47:49,145.0,159.45,163.20,164.65,0.000000,0.000000,1.0,11,2.605472,True,REGULAR,USD
1,MSFT211029C00170000,2021-09-29 15:53:38,170.0,114.20,138.85,139.80,0.000000,0.000000,NaN,8,1.812501,True,REGULAR,USD
2,MSFT211029C00175000,2021-10-12 14:36:35,175.0,128.70,133.30,134.55,0.000000,0.000000,1.0,16,1.945313,True,REGULAR,USD
3,MSFT211029C00180000,2021-09-28 18:35:36,180.0,105.95,128.85,129.85,0.000000,0.000000,NaN,8,1.683595,True,REGULAR,USD
4,MSFT211029C00185000,2021-09-28 18:35:36,185.0,101.00,123.20,124.85,0.000000,0.000000,NaN,8,1.942383,True,REGULAR,USD
5,MSFT211029C00190000,2021-10-15 17:15:28,190.0,119.55,118.85,119.85,0.000000,0.000000,1.0,25,1.531252,True,REGULAR,USD
6,MSFT211029C00195000,2021-10-04 15:24:51,195.0,104.90,113.85,114.85,0.000000,0.000000,1.0,3,1.457034,True,REGULAR,USD
7,MSFT211029C00200000,2021-10-22 19:37:06,200.0,109.38,108.30,109.60,8.079994,7.976302,16.0,11,1.562502,True,REGULAR,USD
8,MSFT211029C00210000,2021-10-12 16:26:13,210.0,92.24,98.70,99.85,0.000000,0.000000,1.0,8,1.167973,True,REGULAR,USD
9,MSFT211029C00215000,2021-09-20 04:15:54,215.0,88.07,93.70,94.85,0.000000,0.000000,NaN,1,1.103520,True,REGULAR,USD
